In [1]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.plotly import interactFigure

In [2]:
T = np.load('data/tick.npz')['BTCUSDT']
len(T)

889360

In [3]:
def model(step: int = (1, 200, 1)):
    rnk = Renko(step, T['PriceA'])
    rnk.create_renko()

    X = pd.DataFrame(rnk.bricks)
    X['mid'] = (X.close + X.open) / 2
    renko = X.mid.values
    return 0, locals()

In [4]:
interactFigure(model, {'renko': {'color': 'gray', 'opacity': 0.4}})